In this notebook, we will cover:

* [Grouped Mutate](#Grouped-Mutate)
* [Separating](#Separating)
* [Uniting](#Uniting)
* [Missing Values](#Missing-Values)

In [33]:
library(tidyverse)

# Grouped Mutate

In the previous lecture we used `mutate` in conjunction with `group_by`. So far, we have used `summarize` in conjunction with `group_by`. In the case of `summarize` we use summary functions (`sum`, `mean`, etc.) that reduce the group's data to a single numerical summary value. For using `mutate` with `group_by`, it is most useful to use window functions instead. You can learn more about window functions [here](https://dplyr.tidyverse.org/articles/window-functions.html).

In [34]:
(people <- tribble(
  ~name,             ~names,  ~values,
  #-----------------|--------|------
  "Phillip Woods",   "age",       45,
  "Phillip Woods",   "height",   186,
  "Phillip Woods",   "age",       50,
  "Jessica Cordero", "age",       37,
  "Jessica Cordero", "height",   156
))

name,names,values
<chr>,<chr>,<dbl>
Phillip Woods,age,45
Phillip Woods,height,186
Phillip Woods,age,50
Jessica Cordero,age,37
Jessica Cordero,height,156


In [35]:
people %>%
    group_by(name, names) %>%
    mutate(row = row_number()) %>%
    ungroup() # to drop the groups in the end

name,names,values,row
<chr>,<chr>,<dbl>,<int>
Phillip Woods,age,45,1
Phillip Woods,height,186,1
Phillip Woods,age,50,2
Jessica Cordero,age,37,1
Jessica Cordero,height,156,1


In [36]:
people %>%
    group_by(name, names) %>%
    summarize(size = n(), .groups = "drop") # the optional argument .groups can control grouping structure in summarize() output

name,names,size
<chr>,<chr>,<int>
Jessica Cordero,age,1
Jessica Cordero,height,1
Phillip Woods,age,2
Phillip Woods,height,1


# Separating

In [37]:
table3

,country,year,rate
,<chr>,<int>,<chr>
1,Afghanistan,1999,745/19987071
2,Afghanistan,2000,2666/20595360
3,Brazil,1999,37737/172006362
4,Brazil,2000,80488/174504898
5,China,1999,212258/1272915272
6,China,2000,213766/1280428583


In [38]:
print(table3)

# A tibble: 6 × 3
  country      year rate             
* <chr>       <int> <chr>            
1 Afghanistan  1999 745/19987071     
2 Afghanistan  2000 2666/20595360    
3 Brazil       1999 37737/172006362  
4 Brazil       2000 80488/174504898  
5 China        1999 212258/1272915272
6 China        2000 213766/1280428583


In [39]:
(separated_table3 <- separate(table3, rate, into = c("cases", "population"), sep = "/"))

country,year,cases,population
<chr>,<int>,<chr>,<chr>
Afghanistan,1999,745,19987071
Afghanistan,2000,2666,20595360
Brazil,1999,37737,172006362
Brazil,2000,80488,174504898
China,1999,212258,1272915272
China,2000,213766,1280428583


In [40]:
print(separated_table3)

# A tibble: 6 × 4
  country      year cases  population
  <chr>       <int> <chr>  <chr>     
1 Afghanistan  1999 745    19987071  
2 Afghanistan  2000 2666   20595360  
3 Brazil       1999 37737  172006362 
4 Brazil       2000 80488  174504898 
5 China        1999 212258 1272915272
6 China        2000 213766 1280428583


The types of the `cases` and `populations` columns are not right. We can change that ourselves.

In [41]:
print(mutate(separated_table3, cases = as.integer(cases), population = as.integer(population)))

# A tibble: 6 × 4
  country      year  cases population
  <chr>       <int>  <int>      <int>
1 Afghanistan  1999    745   19987071
2 Afghanistan  2000   2666   20595360
3 Brazil       1999  37737  172006362
4 Brazil       2000  80488  174504898
5 China        1999 212258 1272915272
6 China        2000 213766 1280428583


Alternatively, we can ask `separate()` to try and convert to a better type.

In [42]:
(separated_table3 <- separate(table3, rate, into = c("cases", "population"), sep = "/", convert = TRUE))

country,year,cases,population
<chr>,<int>,<int>,<int>
Afghanistan,1999,745,19987071
Afghanistan,2000,2666,20595360
Brazil,1999,37737,172006362
Brazil,2000,80488,174504898
China,1999,212258,1272915272
China,2000,213766,1280428583


In [43]:
print(separated_table3)

# A tibble: 6 × 4
  country      year  cases population
  <chr>       <int>  <int>      <int>
1 Afghanistan  1999    745   19987071
2 Afghanistan  2000   2666   20595360
3 Brazil       1999  37737  172006362
4 Brazil       2000  80488  174504898
5 China        1999 212258 1272915272
6 China        2000 213766 1280428583


# Uniting

In [44]:
(separated_year <- separate(table3, year, into = c("century", "year"), sep = 2))

country,century,year,rate
<chr>,<chr>,<chr>,<chr>
Afghanistan,19,99,745/19987071
Afghanistan,20,00,2666/20595360
Brazil,19,99,37737/172006362
Brazil,20,00,80488/174504898
China,19,99,212258/1272915272
China,20,00,213766/1280428583


In [45]:
unite(separated_year, new, century, year)

country,new,rate
<chr>,<chr>,<chr>
Afghanistan,19_99,745/19987071
Afghanistan,20_00,2666/20595360
Brazil,19_99,37737/172006362
Brazil,20_00,80488/174504898
China,19_99,212258/1272915272
China,20_00,213766/1280428583


In [46]:
unite(separated_year, new, century, year, sep = "")

country,new,rate
<chr>,<chr>,<chr>
Afghanistan,1999,745/19987071
Afghanistan,2000,2666/20595360
Brazil,1999,37737/172006362
Brazil,2000,80488/174504898
China,1999,212258/1272915272
China,2000,213766/1280428583


# Missing Values

Missing values can be:

* **Explicit**, or
* **Implicit**

In [47]:
(stocks <- tibble(
  year   = c(2015, 2015, 2015, 2015, 2016, 2016, 2016),
  qtr    = c(   1,    2,    3,    4,    2,    3,    4),
  return = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
))

year,qtr,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


In [48]:
stocks[[4,3]] = 0.3
stocks

year,qtr,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.30
2016,2,0.92
2016,3,0.17
2016,4,2.66


The **complete** command makes implicit missing value explicit by considering all combinations of unique values of specified variables.

In [49]:
complete(stocks, year, qtr) 

year,qtr,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.30
2016,1,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


The missing values also become explicit if we use `pivot_wider()` on the tibble.

In [50]:
pivot_wider(stocks, names_from = year, values_from = return)

qtr,2015,2016
<dbl>,<dbl>,<dbl>
1,1.88,NA
2,0.59,0.92
3,0.35,0.17
4,0.30,2.66


`pivot_longer()` will keep all these explicitly missing values by default.

In [51]:
stocks %>%
    pivot_wider(names_from = year, values_from = return) %>%
    pivot_longer(c(`2015`, `2016`), names_to = "year", values_to = "return")

qtr,year,return
<dbl>,<chr>,<dbl>
1,2015,1.88
1,2016,NA
2,2015,0.59
2,2016,0.92
3,2015,0.35
3,2016,0.17
4,2015,0.30
4,2016,2.66


If you don't like the default behavior of `pivot_longer()`, you can turn off explicit missing values using the `values_drop_na` argument.

In [52]:
stocks %>%
    pivot_wider(names_from = year, values_from = return) %>%
    pivot_longer(c(`2015`, `2016`), names_to = "year", values_to = "return", values_drop_na = TRUE)

qtr,year,return
<dbl>,<chr>,<dbl>
1,2015,1.88
2,2015,0.59
2,2016,0.92
3,2015,0.35
3,2016,0.17
4,2015,0.30
4,2016,2.66


**Imputation** refers to process of filling in missing values. There are many imputation method. One of the simplest ones is *last observation carry forward*. You can do that using the **fill** command.

In [53]:
stocks %>%
    fill(return)

year,qtr,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.30
2016,2,0.92
2016,3,0.17
2016,4,2.66


In [54]:
stocks %>% 
    fill(return, .direction = "up") # next observation carry backwards!

year,qtr,return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.30
2016,2,0.92
2016,3,0.17
2016,4,2.66
